In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import collections
import seaborn as sns
nltk.download('rslp')
from nltk.stem import RSLPStemmer
import heapq as hp
from collections import Counter,OrderedDict
import math

[nltk_data] Downloading package rslp to /home/vinicius/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
data = pd.read_csv('../lab3/results.csv')

In [4]:
#generate tokens from a document
def parse(doc):
    words = ''.join(str(v) for v in doc).lower()
    return RegexpTokenizer(r'[A-zÀ-ú\d]{4,}').tokenize(words)

# produces: term:[(doc, frequency)]
#build an index for a collection of documents
def build_index_with_frequency(docs):
    index = {}
    n = 0
    for text in docs.texto:
        n +=1
        tokens = parse(text)
        for token in tokens:
            if (token in index):
                hasDoc = False
                for i in range(len(index[token])):
                    tup = index[token][i]
                    if (tup[0] == n):
                        index[token][i] = (n, tup[1] + 1)
                        hasDoc = True
                if (not hasDoc):
                    index[token].append((n,1))
            else:
                index[token] = [(n,1)]
    return index
index = build_index_with_frequency(data)

In [25]:
def document_at_time_retrieval(query, index, n_results):
    inverted_lists = []
    results = []
    
    for term in query.split():
        inverted_lists.append(index[term])
    
    documents = []
    for key in index:
        il = index[key]
        for doc in il:
            documents.append(doc[0])
    documents = list(set(documents))
    
    for doc in documents:
        score = 0
        for il in inverted_lists:
            for d in il:
                if (d[0] == doc):
                    score += d[1]
        results.append((score, doc))
    #[(score, doc)]
    results = sorted(results, reverse=True)
    r = []
    for i in range(n_results):
        if(results[i][0] > 0):
            r.append(results[i][1])
    return r

In [6]:
def getDocs(iPosition):
    docs = []
    for el in iPosition:
        docs.append(el[0])
    docs = set(docs)
    return docs

def mutual_info(word, index):
    rank = {}
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = nab/(na*nb)
            
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

In [7]:
def expected_mutual_info(word,index):
    rank = {}
    n = len(data)
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = nab * math.log(n*(nab/(na*nb)))

            
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

In [8]:
def chi_square(word,index):
    rank = {}
    n = len(data)
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = (nab-(1/n)*na*nb)**2 / (na*nb)
     
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

In [9]:
def dice(word,index):
    rank = {}
    n = len(data)
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = nab/(na+nb)
     
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

## Questão 1:

In [10]:
from IPython.display import display, HTML
queries = ["jogo", "partida", "futebol", "campo", "universidade"]
for query in queries:
    results_mim = []
    for word in mutual_info(query, index): 
        results_mim.append(word)
        if len(results_mim) == 10: break
            
    results_emim = []
    for word in expected_mutual_info(query, index): 
        results_emim.append(word)
        if len(results_emim) == 10: break
    
    results_x2 = []        
    for word in chi_square(query, index): 
        results_x2.append(word)
        if len(results_x2) == 10: break
    
    results_dice = []        
    for word in dice(query, index): 
        results_dice.append(word)
        if len(results_dice) == 10: break
    
    df = pd.DataFrame()
    df['MIM'] = list(results_mim)
    df['EMIM'] = list(results_emim)
    df['X2'] = list(results_x2)
    df['DICE'] = list(results_dice)
    print(query)
    display(df)
    print("\n")

jogo


,MIM,EMIM,X2,DICE
0,atacante,bola,bola,bola
1,profissão,futebol,atacante,futebol
2,sterling,área,oitavas,área
3,yaya,jogador,champions,campo
4,touré,champions,futebol,real
5,kevin,atacante,jogador,jogador
6,prince,oitavas,juve,primeiro
7,boateng,campo,quartas,partida
8,ballotelli,gols,torneio,time
9,lilian,jogadores,atlético,gols




partida


,MIM,EMIM,X2,DICE
0,profissão,transmitida,transmitida,transmitida
1,sterling,horário,horário,horário
2,yaya,jogo,enfrentam,enfrentam
3,touré,enfrentam,esporte,jogo
4,kevin,esporte,torcida,jogador
5,boateng,jogador,juve,confronto
6,ballotelli,confronto,atlético,esporte
7,lilian,ganhar,bater,ganhar
8,thuram,volta,estádio,futebol
9,insulto,estádio,barça,brasília




futebol


,MIM,EMIM,X2,DICE
0,profissão,jogador,jogador,jogador
1,sterling,treinador,treinador,treinador
2,yaya,jogadores,atleta,jogadores
3,touré,seleção,jogadores,atleta
4,kevin,atleta,campeonato,seleção
5,boateng,bola,clubes,bola
6,ballotelli,equipe,torcedores,clubes
7,lilian,jogo,seleção,campeonato
8,thuram,clubes,seleções,time
9,insulto,time,categorias,jogos




campo


,MIM,EMIM,X2,DICE
0,profissão,jogo,chegada,jogo
1,sterling,seleção,articulação,seleção
2,yaya,chegada,seleção,sempre
3,touré,sempre,procedentes,enquanto
4,kevin,enquanto,competição,chegada
5,boateng,futebol,impressionantes,futebol
6,ballotelli,primeiro,completar,primeiro
7,lilian,articulação,residentes,todas
8,thuram,todas,bolívia,importante
9,insulto,importante,pedem,mulheres




universidade


,MIM,EMIM,X2,DICE
0,carolina,professor,antropóloga,professor
1,levantam,muitos,professor,políticas
2,pátria,políticas,pesquisadora,muitos
3,relaxam,exemplo,bairro,exemplo
4,flui,pesquisadora,estudantes,cidades
5,baixam,cidades,pesquisador,século
6,vinham,antropóloga,william,comum
7,bolos,conceito,harvard,esses
8,chocolates,foram,planned,eram
9,ortegano,esses,parenthood,conceito


A partir dos resultados apresentados, acredito que a chi square tenha obtido melhores resultados. 
Para os 4 primeiros termos, o resultado é muito semelhante. No entanto, observando o ultimo, referenque à query `universidade` vemos que a chi square retorna valores que complementam melhor a busca do que as outras métricas.

In [11]:
### Questão 2: 

A query inicial escolhida é `corrupção`. 
Para avaliar a qualidade das consultas, iremos observar inicialmente seu título e subtílo e verificar se o que está sendo apresentado realmente tem a ver com `corrupção`. Caso necessário, iremos para a url da notícia, para avaliar essa relação.

In [51]:
#data.url[100]
#data.titulo[100]
#data.subtitulo[100]
pd.options.display.max_colwidth = 500
def print_doc_at_time_retrieval_docs(q, i, n):
    docs = document_at_time_retrieval(q, i, n)
    results_title = []
    results_subtitle = []
    results_url = []
    for doc in docs:
        doc_index = doc-1
        results_title.append(data.titulo[doc_index])
        results_subtitle.append(data.subtitulo[doc_index])
        results_url.append(data.url[doc_index])
    df_query_0 = pd.DataFrame()
    df_query_0["title"] = list(results_title)
    df_query_0["subtitle"] = list(results_subtitle)
    df_query_0["url"] = list(results_url)
    display(df_query_0)
    print("\n")

def get_n_closest_terms_x2(q, index, n):
    results_x2 = []        
    for word in chi_square(q, index): 
        results_x2.append(word)
        if len(results_x2) == n: break
    return results_x2

**Apenas a query original:**

In [52]:
query = "corrupção"
print("query: " + query)
print_doc_at_time_retrieval_docs(query, index, 10)

query: corrupção


,title,subtitle,url
0,A história não pode ser apagada ou reescrita. Lembrar que houve ditadura valoriza as conquistas de hoje,"O regime militar merece ser recordado, mas não pelos motivos errados. Reconhecer o passado enaltece conquistas sociais, como o próprio combate à corrupção",https://brasil.elpais.com/brasil/2019/03/30/opinion/1553972319_896574.html
1,"FMI descarta recessão mundial, mas alerta para o impacto do Brexit e da guerra comercial",Christine Lagarde afirma que crescimento global é vulnerável e a desaceleração é mais pronunciada do que se previa,https://brasil.elpais.com/brasil/2019/04/02/economia/1554201957_597143.html
2,O breve discurso de Bolsonaro decepciona em Davos,"Falta de detalhes sobre medidas concretas gera frustração na audiência do Fórum. “Foi um pouco decepcionante, mas a verdade é que as metas que fixou foram alentadoras”, avaliava o economista-chefe da seguradora de riscos IHS",https://brasil.elpais.com/brasil/2019/01/22/economia/1548182020_953667.html
3,Bolsonaro (des)governa o Brasil pelo Twitter,"Ao tomar decisões pelo volume dos gritos nas redes sociais, o presidente corrompe a democracia",https://brasil.elpais.com/brasil/2019/03/06/opinion/1551904505_351681.html
4,A morte do inocente neto de Lula soltou os monstros do ódio,"Cai sobre nossa consciência de adultos a infâmia de transformar em piadas baratas, em ironia e sarcasmo a dor de um avô pela perda de seu neto",https://brasil.elpais.com/brasil/2019/03/02/opinion/1551487708_675741.html
5,Chicago elege sua primeira prefeita negra e homossexual,"Lori Lightfoot supera Toni Preckwinkle, também afro-americana, por quase 50 pontos",https://brasil.elpais.com/brasil/2019/04/03/internacional/1554261437_488996.html
6,Esportes vivem paralisia sem planos detalhados e nomeações travadas,"Rebaixada de ministério a secretaria, política esportiva de Bolsonaro espelha modelo do regime ditatorial, com militares na linha de frente, engessada pela falta de autonomia",https://brasil.elpais.com/brasil/2019/03/30/deportes/1553901938_175156.html
7,As duas faces de um Brasil entediante,"Com treinadores pressionados antes de competições importantes, as seleções masculina e feminina de futebol capricham no discurso, mas decepcionam na prática",https://brasil.elpais.com/brasil/2019/03/26/deportes/1553634637_125842.html
8,Guia para não passar vergonha quando falar de ditadura,"Em meio ao fogo cruzado do debate negacionista do regime, uma lista de livros, filmes e documentos com informações essenciais sobre o regime militar",https://brasil.elpais.com/brasil/2019/04/01/cultura/1554136024_994794.html
9,Moro é mais popular que Bolsonaro em Governo que segue perdendo aprovação,Atlas Político confirma tendência de queda na popularidade da gestão e do presidente e mostra índice bom/excelente numericamente atrás de regular e ruim/péssimo. 87% são a favor da prisão de Temer,https://brasil.elpais.com/brasil/2019/04/03/politica/1554324591_827255.html


**A query expandida em 3 termos:**

In [53]:
terms = get_n_closest_terms_x2(query, index, 3)
query_3 = query
for term in terms:
    query_3 += " " + term
print("query: " + query_3)
print_doc_at_time_retrieval_docs(query_3, index, 10)

query: corrupção envolvendo alves seguidamente


,title,subtitle,url
0,A história não pode ser apagada ou reescrita. Lembrar que houve ditadura valoriza as conquistas de hoje,"O regime militar merece ser recordado, mas não pelos motivos errados. Reconhecer o passado enaltece conquistas sociais, como o próprio combate à corrupção",https://brasil.elpais.com/brasil/2019/03/30/opinion/1553972319_896574.html
1,Quem mandou matar Marielle? E por quê?,"Bolsonaro, que governa o Brasil pela administração do ódio, deveria ser o maior interessado em desvendar o crime",https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html
2,Bolsonaro (des)governa o Brasil pelo Twitter,"Ao tomar decisões pelo volume dos gritos nas redes sociais, o presidente corrompe a democracia",https://brasil.elpais.com/brasil/2019/03/06/opinion/1551904505_351681.html
3,"FMI descarta recessão mundial, mas alerta para o impacto do Brexit e da guerra comercial",Christine Lagarde afirma que crescimento global é vulnerável e a desaceleração é mais pronunciada do que se previa,https://brasil.elpais.com/brasil/2019/04/02/economia/1554201957_597143.html
4,“Lógica de usar torturadores da ditadura no crime foi usada nas milícias”,"Aloy Jupiara, coautor de 'Os porões da contravenção', fala sobre o legado de profissionalização do crime deixado pelos anos de chumbo que perdura até hoje",https://brasil.elpais.com/brasil/2019/03/29/politica/1553885098_115676.html
5,Bolsonaro manda festejar o crime,"Ao determinar a comemoração do golpe militar de 1964, o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça",https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html
6,Esportes vivem paralisia sem planos detalhados e nomeações travadas,"Rebaixada de ministério a secretaria, política esportiva de Bolsonaro espelha modelo do regime ditatorial, com militares na linha de frente, engessada pela falta de autonomia",https://brasil.elpais.com/brasil/2019/03/30/deportes/1553901938_175156.html
7,O breve discurso de Bolsonaro decepciona em Davos,"Falta de detalhes sobre medidas concretas gera frustração na audiência do Fórum. “Foi um pouco decepcionante, mas a verdade é que as metas que fixou foram alentadoras”, avaliava o economista-chefe da seguradora de riscos IHS",https://brasil.elpais.com/brasil/2019/01/22/economia/1548182020_953667.html
8,Um tuíte muito vulgar,Bolsonaro lança cortina de fumaça nas redes sociais em detrimento da imagem institucional do Brasil,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551926876_408228.html
9,A morte do inocente neto de Lula soltou os monstros do ódio,"Cai sobre nossa consciência de adultos a infâmia de transformar em piadas baratas, em ironia e sarcasmo a dor de um avô pela perda de seu neto",https://brasil.elpais.com/brasil/2019/03/02/opinion/1551487708_675741.html


**A query expandida em 5 termos:**

In [54]:
terms = get_n_closest_terms_x2(query, index, 5)
query_5 = query
for term in terms:
    query_5 += " " + term
print("query: " + query_5)
print_doc_at_time_retrieval_docs(query_5, index, 10)

query: corrupção envolvendo alves seguidamente ligação cidadania


,title,subtitle,url
0,A história não pode ser apagada ou reescrita. Lembrar que houve ditadura valoriza as conquistas de hoje,"O regime militar merece ser recordado, mas não pelos motivos errados. Reconhecer o passado enaltece conquistas sociais, como o próprio combate à corrupção",https://brasil.elpais.com/brasil/2019/03/30/opinion/1553972319_896574.html
1,Quem mandou matar Marielle? E por quê?,"Bolsonaro, que governa o Brasil pela administração do ódio, deveria ser o maior interessado em desvendar o crime",https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html
2,Esportes vivem paralisia sem planos detalhados e nomeações travadas,"Rebaixada de ministério a secretaria, política esportiva de Bolsonaro espelha modelo do regime ditatorial, com militares na linha de frente, engessada pela falta de autonomia",https://brasil.elpais.com/brasil/2019/03/30/deportes/1553901938_175156.html
3,“Lógica de usar torturadores da ditadura no crime foi usada nas milícias”,"Aloy Jupiara, coautor de 'Os porões da contravenção', fala sobre o legado de profissionalização do crime deixado pelos anos de chumbo que perdura até hoje",https://brasil.elpais.com/brasil/2019/03/29/politica/1553885098_115676.html
4,Bolsonaro manda festejar o crime,"Ao determinar a comemoração do golpe militar de 1964, o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça",https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html
5,Bolsonaro (des)governa o Brasil pelo Twitter,"Ao tomar decisões pelo volume dos gritos nas redes sociais, o presidente corrompe a democracia",https://brasil.elpais.com/brasil/2019/03/06/opinion/1551904505_351681.html
6,"FMI descarta recessão mundial, mas alerta para o impacto do Brexit e da guerra comercial",Christine Lagarde afirma que crescimento global é vulnerável e a desaceleração é mais pronunciada do que se previa,https://brasil.elpais.com/brasil/2019/04/02/economia/1554201957_597143.html
7,O breve discurso de Bolsonaro decepciona em Davos,"Falta de detalhes sobre medidas concretas gera frustração na audiência do Fórum. “Foi um pouco decepcionante, mas a verdade é que as metas que fixou foram alentadoras”, avaliava o economista-chefe da seguradora de riscos IHS",https://brasil.elpais.com/brasil/2019/01/22/economia/1548182020_953667.html
8,Um tuíte muito vulgar,Bolsonaro lança cortina de fumaça nas redes sociais em detrimento da imagem institucional do Brasil,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551926876_408228.html
9,A morte do inocente neto de Lula soltou os monstros do ódio,"Cai sobre nossa consciência de adultos a infâmia de transformar em piadas baratas, em ironia e sarcasmo a dor de um avô pela perda de seu neto",https://brasil.elpais.com/brasil/2019/03/02/opinion/1551487708_675741.html


**A query expandida em 10 termos:**

In [55]:
terms = get_n_closest_terms_x2(query, index, 10)
query_10 = query
for term in terms:
    query_10 += " " + term
print("query: " + query_10)
print_doc_at_time_retrieval_docs(query_10, index, 10)

query: corrupção envolvendo alves seguidamente ligação cidadania potência federal denúncias milícia repórter


,title,subtitle,url
0,Quem mandou matar Marielle? E por quê?,"Bolsonaro, que governa o Brasil pela administração do ódio, deveria ser o maior interessado em desvendar o crime",https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html
1,“Lógica de usar torturadores da ditadura no crime foi usada nas milícias”,"Aloy Jupiara, coautor de 'Os porões da contravenção', fala sobre o legado de profissionalização do crime deixado pelos anos de chumbo que perdura até hoje",https://brasil.elpais.com/brasil/2019/03/29/politica/1553885098_115676.html
2,Bolsonaro (des)governa o Brasil pelo Twitter,"Ao tomar decisões pelo volume dos gritos nas redes sociais, o presidente corrompe a democracia",https://brasil.elpais.com/brasil/2019/03/06/opinion/1551904505_351681.html
3,A história não pode ser apagada ou reescrita. Lembrar que houve ditadura valoriza as conquistas de hoje,"O regime militar merece ser recordado, mas não pelos motivos errados. Reconhecer o passado enaltece conquistas sociais, como o próprio combate à corrupção",https://brasil.elpais.com/brasil/2019/03/30/opinion/1553972319_896574.html
4,Esportes vivem paralisia sem planos detalhados e nomeações travadas,"Rebaixada de ministério a secretaria, política esportiva de Bolsonaro espelha modelo do regime ditatorial, com militares na linha de frente, engessada pela falta de autonomia",https://brasil.elpais.com/brasil/2019/03/30/deportes/1553901938_175156.html
5,Bolsonaro manda festejar o crime,"Ao determinar a comemoração do golpe militar de 1964, o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça",https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html
6,"Milícia, o exército popular a serviço de Maduro","O poder do líder bolivariano na Venezuela se baseia no exército e nas milícias, um corpo de um milhão de pessoas treinadas com vassouras e rifles, que prometem dar suas vidas pela Revolução",https://brasil.elpais.com/brasil/2019/03/29/internacional/1553824596_128119.html
7,Não temos tempo a perder. A hora de unir as forças democráticas é agora,Essa articulação requererá deixar de lado diferenças e firmar um compromisso pela defesa do interesse da sociedade brasileira como um todo,https://brasil.elpais.com/brasil/2019/04/03/opinion/1554293843_323322.html
8,"FMI descarta recessão mundial, mas alerta para o impacto do Brexit e da guerra comercial",Christine Lagarde afirma que crescimento global é vulnerável e a desaceleração é mais pronunciada do que se previa,https://brasil.elpais.com/brasil/2019/04/02/economia/1554201957_597143.html
9,Guia para não passar vergonha quando falar de ditadura,"Em meio ao fogo cruzado do debate negacionista do regime, uma lista de livros, filmes e documentos com informações essenciais sobre o regime militar",https://brasil.elpais.com/brasil/2019/04/01/cultura/1554136024_994794.html


Pelos títulos e subtítulos dos documentos recuperados, podemos observar que conforme fomos expandindo a consulta com mais termos, as notícias foram ficando cada vez mais focadas no governo brasileiro. 
Isso de certa forma faz sentido e pode mostrar um ganho de precisão pois um dos termos, se não o termo mais associado à política brasileira é `corrupção`. 
Ao abrir os links e ler as notícias, pude cosntatar também que foram crescendo o número de documentos em que o assunto tratado era diretamente relacionado à corrupção cresceu, ao passo que os que ele era tratado indiretamente diminuiu.
Assim, podemos dizer que `a qualidade dos resultados aumentou significativamente`.